# Load libraries

In [1]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np

C:\Users\Kat\anaconda3\envs\tensor\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Kat\anaconda3\envs\tensor\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Kat\anaconda3\envs\tensor\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Kat\anaconda3\envs\tensor\lib\site-packages\tensorflow\python\frame

# Load data

In [2]:
blue=pd.read_csv('fad_blue_rnadata.csv').rename(columns={"Unnamed: 0": "Cells"})
blue.set_index("Cells", drop=True,inplace=True)
blue=blue.T
#^transpose because we want genes to be features

In [3]:
idx=blue.index 
Y=blue['BlueFP']
x=blue.drop(['BlueFP'],axis=1)
#get top 200 genes from linear reg
genestokeep=pd.read_csv('data/top200_genes.csv').drop("Unnamed: 0",axis=1)['Genes']
x2=x.loc[:, x.columns.isin(genestokeep)]
genes=x2.columns
#set x and y
X=x2.to_numpy()

# Try different model architectures

Referenced this tutorial: https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/

In [4]:
def small_model():
    # create model
    model = Sequential()
    model.add(Dense(X.shape[1], input_dim=X.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [5]:
def deeper_model():
    # create model
    model = Sequential()
    model.add(Dense(X.shape[1], input_dim=X.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dense(50, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [6]:
def wider_model():
    # create model
    model = Sequential()
    model.add(Dense(250, input_dim=X.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [7]:
# evaluate model  
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=small_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print('Small model: ')
print("Standardized: %.2f (%.2f) RMSE" % (np.mean(np.sqrt(np.absolute(results))), np.mean(np.sqrt(np.absolute(results)))))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Small model: 
Standardized: 6.71 (6.71) RMSE


In [8]:
# evaluate model with  
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=deeper_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print('Deeper model: ')
print("Standardized: %.2f (%.2f) RMSE" % (np.mean(np.sqrt(np.absolute(results))), np.mean(np.sqrt(np.absolute(results)))))

Deeper model: 
Standardized: 6.26 (6.26) RMSE


In [9]:
# evaluate model  
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print('Wider model: ')
print("Standardized: %.2f (%.2f) RMSE" % (np.mean(np.sqrt(np.absolute(results))), np.mean(np.sqrt(np.absolute(results)))))

Wider model: 
Standardized: 6.75 (6.75) RMSE


In [10]:
def deeper_model2():
    # create model
    model = Sequential()
    model.add(Dense(X.shape[1], input_dim=X.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dense(150, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [11]:
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=deeper_model2, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print('Deeper model 2: ')
print("Standardized: %.2f (%.2f) RMSE" % (np.mean(np.sqrt(np.absolute(results))), np.mean(np.sqrt(np.absolute(results)))))

Deeper model 2: 
Standardized: 6.17 (6.17) RMSE


In [12]:
def deepest_model():
    # create model
    model = Sequential()
    model.add(Dense(X.shape[1], input_dim=X.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dense(50, kernel_initializer='normal', activation='relu'))
    model.add(Dense(150, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [13]:
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=deepest_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print('Deepest model: ')
print("Standardized: %.2f (%.2f) RMSE" % (np.mean(np.sqrt(np.absolute(results))), np.mean(np.sqrt(np.absolute(results)))))

Deepest model: 
Standardized: 6.11 (6.11) RMSE
